In [1]:
'''
what is descriptor in python? why we need Descriptor in python ? lets take a our path to Pythonic descriptor

In general, a descriptor is an object attribute with “binding behavior”, one whose attribute access has been overridden
by methods in the descriptor protocol. Those methods are __get__(), __set__(), and __delete__(). If any of those methods are
defined for an object, it is said to be a descriptor.
'''

'\nwhat is descriptor in python? why we need Descriptor in python ? lets take a our path to Pythonic descriptor\n\n'

In [6]:
# why we need Descriptor lets dive into real world application

class Order:
    def __init__(self, name, price, quantity):
        self.name = name
        self.price = price
        self.quantity = quantity
        
    def total(self):
        return self.price * self.quantity

apple_order = Order('apple',1,10)
apple_order.total()

# seems straightforward right but just look logically there a bug
apple_order.quantity = -10
apple_order.total()

In [11]:
# yehh look in real world our quantity never go negative thats the problem here 
# lets solve this using descriptor model

class Order:
    def __init__(self, name, price, quantity):
        self._name = name
        self.price = price
        self._quantity = quantity  # (1)

    @property
    def quantity(self):
        return self._quantity

    @quantity.setter
    def quantity(self, value):
        if value < 0:
            raise ValueError('Cannot be negative.')
        self._quantity = value  # (2)
        
apple_order = Order('apple',1,10)
apple_order.quantity = -10
# ValueError: Cannot be negative
apple_order.quantity

ValueError: Cannot be negative.

In [39]:
# yay it now cannot be nagative but wait .....
# other than quantity price can also be aassigned negative value 
# what about that ...........

# since we know that if we have same portion of code for two different operation 
# we would lik to make a single reusable code Right! 
# so lets start from different way 

class NonNegative:
    def __init__(self, name):
        self.name = name  # (4)
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]  # (5)
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Cannot be negative.')
        instance.__dict__[self.name] = value  # (6)


class Order:
    price = NonNegative('price')  # (3)
    quantity = NonNegative('quantity')

    def __init__(self, name, price, quantity):
        self._name = name
        self.price = price
        self.quantity = quantity

    def total(self):
        return self.price * self.quantity

apple_order = Order('apple', 1, 10)
apple_order.total()
# 10
# apple_order.price = -10
# ValueError: Cannot be negative
apple_order.quantity = 10
apple_order.quantity

10

In [2]:
# now same thing can be achieved using python new descriptor Protocol

class NonNegative:
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Cannot be negative.')
        instance.__dict__[self.name] = value
    def __set_name__(self, owner, name):
        print(str(owner))
        print(name)
        self.name = name

class Order:
    price = NonNegative()
    quantity = NonNegative()

    def __init__(self, name, price, quantity):
        self._name = name
        self.price = price
        self.quantity = quantity

    def total(self):
        return self.price * self.quantity

apple_order = Order('apple', 1, 10)
print(apple_order.total())
# 10
apple_order.price = -10
# ValueError: Cannot be negative
apple_order.quantity = -10
# ValueError: Cannot be negative

<class '__main__.Order'>
price
<class '__main__.Order'>
quantity
10


ValueError: Cannot be negative.

In [41]:
# yalap yalap yalap